In [1]:
# bibliotecas
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import date
import joblib

In [2]:
# cria função para configurar colunas de bb
def cria_bb( ticker, inicio, fim ):
    # coleta de dados
    data = pdr.get_data_yahoo( ticker, start= inicio, end=fim )

    # criando uma coluna contentdo média móvel com 20 períodos
    data.loc[ :, "MM" ] = data[ 'Adj Close' ].rolling( 20 ).mean()

    # criando uma coluna contendo os desvios de 20 períodos
    data.loc[ :, "desvio" ] = data[ 'Adj Close' ].rolling( 20 ).std()

    # cria colunas contedos valores da banda superior e banda inferior
    data.loc[ :, "Banda_sup" ] = data.MM + ( data.desvio * 2 )
    data.loc[ :, "Banda_inf" ] = data.MM - ( data.desvio * 2 )

    # excluindo linhas faltantes
    data.dropna(inplace=True)

    # criando valores para sinal inferior e superior
    data.loc[ :, "Sinal_inf" ] = np.where(
        ( data.loc[ :, "Close" ] < data.loc[ :, "Banda_inf" ] ), data.loc[ :, "Close" ] * 0.99, np.nan
    )
    data.loc[:, "Sinal_sup"] = np.where(
        ( data.loc[ :, "Close" ] > data.loc[ :, "Banda_sup" ] ), data.loc[ :, "Close" ] * 1.01, np.nan
    )

    # persistindo em disco com joblib
    joblib.dump( data, "data_amzn" )
    
    return data.head()

In [3]:
ticker = 'AMZN'
inicio = "2015-01-01"
fim  = "2021-06-17"
cria_bb( ticker, inicio, fim )

,High,Low,Open,Close,Volume,Adj Close,MM,desvio,Banda_sup,Banda_inf,Sinal_inf,Sinal_sup
Date,,,,,,,,,,,,
2015-01-30,359.500000,340.739990,346.320007,354.529999,23856100,354.529999,302.747501,14.525972,331.799444,273.695557,NaN,358.075299
2015-02-02,365.000000,350.010010,350.049988,364.470001,10231900,364.470001,305.545001,20.037988,345.620978,265.469025,NaN,368.114701
2015-02-03,367.820007,360.209991,360.290009,363.549988,6212600,363.549988,308.613000,23.834924,356.282849,260.943152,NaN,367.185488
2015-02-04,367.500000,358.230011,358.380005,364.750000,4169900,364.750000,312.086000,26.681942,365.449884,258.722116,NaN,NaN
2015-02-05,378.799988,365.899994,366.000000,373.890015,7247600,373.890015,315.859500,29.801789,375.463078,256.255923,NaN,NaN
